# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [1]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [2]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [3]:
earthquake_json['metadata']

{'generated': 1643917043000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2022-01-03&endtime=2022-02-02',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.13.2',
 'count': 10351}

Each element in the JSON array `features` is a row of data for our dataframe.

In [4]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [5]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.95,
  'place': '1 km WNW of Cole, Oklahoma',
  'time': 1643759312549,
  'updated': 1643819696878,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ok2022cgls',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ok2022cgls&format=geojson',
  'felt': 2,
  'cdi': 2.7,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 59,
  'net': 'ok',
  'code': '2022cgls',
  'ids': ',ok2022cgls,',
  'sources': ',ok,',
  'types': ',dyfi,origin,phase-data,',
  'nst': 55,
  'dmin': 0.06298763968,
  'rms': 0.27,
  'gap': 38,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 2.0 - 1 km WNW of Cole, Oklahoma'},
 'geometry': {'type': 'Point',
  'coordinates': [-97.58533333, 35.10766667, 6.78]},
 'id': 'ok2022cgls'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [6]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.95,"1 km WNW of Cole, Oklahoma",1643759312549,1643819696878,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,2.0,2.7,NaN,...,",ok2022cgls,",",ok,",",dyfi,origin,phase-data,",55.0,0.062988,0.270,38.00,ml,earthquake,"M 2.0 - 1 km WNW of Cole, Oklahoma"
1,0.40,"16 km NNW of Sutcliffe, Nevada",1643758895187,1643769518163,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nn00832999,",",nn,",",origin,phase-data,",4.0,0.093000,0.182,165.98,ml,earthquake,"M 0.4 - 16 km NNW of Sutcliffe, Nevada"
2,1.49,"7km W of Palomar Observatory, CA",1643758877630,1643764377670,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ci39930231,",",ci,",",focal-mechanism,nearby-cities,origin,phase-da...",49.0,0.066000,0.190,30.00,ml,earthquake,"M 1.5 - 7km W of Palomar Observatory, CA"
3,1.47,Puerto Rico region,1643758273200,1643759611930,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",pr71332143,",",pr,",",origin,phase-data,",3.0,NaN,0.050,256.00,md,earthquake,M 1.5 - Puerto Rico region
4,2.05,"4 km S of Indios, Puerto Rico",1643758241850,1643759100610,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",pr71332138,",",pr,",",origin,phase-data,",15.0,NaN,0.070,170.00,md,earthquake,"M 2.1 - 4 km S of Indios, Puerto Rico"


### (Optional) Write Data to CSV

In [7]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>